<a href="https://colab.research.google.com/github/sindgisrishti/Rebase_repo/blob/main/colab_granite_demo_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi


Wed Nov  5 13:46:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!python -m pip install --upgrade pip
!pip install transformers accelerate bitsandbytes sentencepiece safetensors gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 38.4 MB/s  0:00:01


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

MODEL = "ibm-granite/granite-3.2-2b-instruct"  # keep 2B version (GPU can handle it)

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map="auto",             # ensures GPU placement
    torch_dtype=torch.float16,     # efficient half-precision
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

gen = pipeline("text-generation", model=model, tokenizer=tokenizer)
print("✅ Granite model loaded on GPU:", torch.cuda.get_device_name(0))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Granite model loaded on GPU: Tesla T4


In [4]:
import pandas as pd, os

os.makedirs("data", exist_ok=True)

data = [
    {"drug_a": "ibuprofen", "drug_b": "warfarin", "severity": "major",
     "description": "Increases bleeding risk due to platelet inhibition and displacement of warfarin from protein binding sites.",
     "safer_alternative": "acetaminophen"},
    {"drug_a": "amoxicillin", "drug_b": "allopurinol", "severity": "moderate",
     "description": "Concurrent use may increase risk of skin rash or hypersensitivity reactions.",
     "safer_alternative": "cephalexin"},
    {"drug_a": "lisinopril", "drug_b": "potassium chloride", "severity": "major",
     "description": "Combining may cause dangerous hyperkalemia (high potassium levels).",
     "safer_alternative": "amlodipine"},
    {"drug_a": "metformin", "drug_b": "alcohol", "severity": "moderate",
     "description": "Alcohol can increase the risk of lactic acidosis in metformin users.",
     "safer_alternative": "none"},
    {"drug_a": "paracetamol", "drug_b": "ibuprofen", "severity": "safe",
     "description": "No significant interaction. Combination commonly used for pain relief.",
     "safer_alternative": "none"},
]

pd.DataFrame(data).to_csv("data/drug_interactions.csv", index=False)
print("✅ Dataset ready.")


✅ Dataset ready.


In [12]:
# ======================================================
# 🩺 MediGuard AI — Prescription Verification System
# Enhanced Hackathon Version — Granite + Rule Engine + PDF Reports
# Author: Guided by ChatGPT (Senior AI Engineer)
# ======================================================

import gradio as gr
import pandas as pd
import json
import torch
import re
import os
from datetime import datetime
import matplotlib.pyplot as plt
import io

try:
    from fpdf import FPDF
except ImportError:
    !pip install fpdf
    from fpdf import FPDF

# ------------------------------------------------------
# ✅ Verify model pipeline exists
# ------------------------------------------------------
try:
    _ = gen("ready check", max_new_tokens=3)
    print("✅ Granite pipeline detected and ready.")
except NameError:
    raise SystemExit("❌ Load your Granite model first (see setup cell above).")

# ------------------------------------------------------
# ✅ Load Dataset
# ------------------------------------------------------
DATA_PATH = "data/drug_interactions.csv"
if not os.path.exists(DATA_PATH):
    raise SystemExit(f"❌ Dataset not found at {DATA_PATH}. Please recreate it first.")

df = pd.read_csv(DATA_PATH)
df['drug_a'] = df['drug_a'].str.lower()
df['drug_b'] = df['drug_b'].str.lower()

# ------------------------------------------------------
# Helper: JSON & regex extraction utilities
# ------------------------------------------------------

def _normalize_med_list(meds):
    out = []
    if not isinstance(meds, list): return out
    for m in meds:
        if not isinstance(m, dict): continue
        name = m.get("name") or m.get("drug") or m.get("medication") or ""
        dosage = m.get("dosage", "")
        freq = m.get("frequency", "")
        name = str(name).strip()
        if name:
            out.append({"name": name, "dosage": str(dosage).strip(), "frequency": str(freq).strip()})
    return out


def _regex_fallback_extract(text):
    """Regex-based extraction if JSON parsing fails"""
    patterns = [
        r"([A-Za-z][A-Za-z\-\d]{1,40})\s+(\d+(?:\.\d+)?\s*(?:mg|mcg|g|ml|IU))\s*(?:\b(OD|BD|TID|QID|daily|once daily|twice daily|three times daily|prn)\b)?",
        r"([A-Za-z][A-Za-z\-\d]{1,40})\s+(\d+(?:\.\d+)?\s*(?:mg|mcg|g|ml|IU))",
        r"([A-Za-z][A-Za-z\-\d]{1,40})\s+(OD|BD|TID|QID|daily|once daily|twice daily|prn)"
    ]
    meds = []
    found = set()
    for pat in patterns:
        for match in re.finditer(pat, text, re.IGNORECASE):
            name = match.group(1).strip()
            dose = match.group(2).strip() if match.lastindex >= 2 and match.group(2) else ""
            freq = match.group(3).strip() if match.lastindex >= 3 and match.group(3) else ""
            key = (name.lower(), dose.lower(), freq.lower())
            if key not in found:
                found.add(key)
                meds.append({"name": name, "dosage": dose, "frequency": freq})
    return meds


def extract_medications(text: str):
    """LLM + Repair + Regex fallback JSON extractor (fixed version)"""
    base_prompt = f"""
    Extract structured medication information from this prescription text.
    Return ONLY valid JSON with this exact schema:
    {{"medications":[{{"name":"DrugName","dosage":"DosageAmount","frequency":"Frequency"}}]}}
    Text: {text}
    """
    try:
        out1 = gen(base_prompt, max_new_tokens=300, do_sample=False)[0]["generated_text"]

        # ✅ FIX: Get LAST JSON block (ignore example schema at top)
        json_matches = list(re.finditer(r"\{(?:[^{}]|\{[^{}]*\})*\}", out1, re.DOTALL))
        if json_matches:
            json_str = json_matches[-1].group(0)
            js = json.loads(json_str)
            meds = _normalize_med_list(js.get("medications", []))
            if meds: return meds

        # Try repair if still invalid
        repair_prompt = f"Fix this to valid JSON using same schema only:\n{out1}"
        out2 = gen(repair_prompt, max_new_tokens=200, do_sample=False)[0]["generated_text"]
        json_matches2 = list(re.finditer(r"\{(?:[^{}]|\{[^{}]*\})*\}", out2, re.DOTALL))
        if json_matches2:
            json_str2 = json_matches2[-1].group(0)
            js2 = json.loads(json_str2)
            meds = _normalize_med_list(js2.get("medications", []))
            if meds: return meds

        # Final fallback: regex extractor
        fallback = _regex_fallback_extract(text)
        if fallback:
            return fallback

        return [{"name":"Parsing error","dosage":"No valid JSON","frequency":""}]
    except Exception as e:
        return [{"name":"Parsing error","dosage":str(e),"frequency":""}]


# ------------------------------------------------------
# Drug interaction + analysis logic
# ------------------------------------------------------

def check_interactions(meds):
    interactions = []
    names = [m["name"].lower() for m in meds]
    for i in range(len(names)):
        for j in range(i+1, len(names)):
            a, b = names[i], names[j]
            match = df[((df['drug_a']==a)&(df['drug_b']==b))|((df['drug_a']==b)&(df['drug_b']==a))]
            if not match.empty:
                row = match.iloc[0]
                interactions.append({
                    "pair": f"{a} + {b}",
                    "severity": row["severity"],
                    "description": row["description"],
                    "safer_alternative": row["safer_alternative"]
                })
            else:
                interactions.append({
                    "pair": f"{a} + {b}",
                    "severity": "unknown",
                    "description": "No documented interaction found.",
                    "safer_alternative": "none"
                })
    return interactions


def analyze_with_granite(age, meds, interactions):
    meds_str = ", ".join([f"{m['name']} ({m.get('dosage','')})" for m in meds])
    prompt = f"""
    You are an expert clinical pharmacist.
    A {age}-year-old patient is prescribed: {meds_str}.
    Interactions: {json.dumps(interactions, indent=2)}.
    Write a concise summary with:
    - Risk level
    - Key interactions and reasons
    - Safer alternatives
    - Final advice (2–3 lines)
    """
    try:
        out = gen(prompt, max_new_tokens=250, do_sample=False)[0]["generated_text"]
        return out.strip()
    except Exception as e:
        return f"Error generating summary: {e}"


def compute_confidence(interactions):
    base = 0.95
    penalty = 0.05 * len([i for i in interactions if i["severity"] in ["major","moderate"]])
    return round(max(0.75, base - penalty), 2)


# ------------------------------------------------------
# Visualization & PDF generator
# ------------------------------------------------------

def make_chart(interactions, path):
    counts = {"major":0,"moderate":0,"safe":0,"unknown":0}
    for it in interactions: counts[it["severity"]] = counts.get(it["severity"],0)+1
    labels = [f"{k.capitalize()} ({v})" for k,v in counts.items() if v>0]
    sizes = [v for v in counts.values() if v>0]
    if not sizes: sizes, labels = [1], ["No Interactions"]
    plt.figure(figsize=(4,3))
    plt.pie(sizes, labels=labels, autopct="%1.0f%%")
    plt.title("Interaction Severity Distribution")
    plt.tight_layout()
    plt.savefig(path, dpi=150)
    plt.close()


def generate_pdf(report_json):
    report = json.loads(report_json)
    pdf_path = f"/tmp/mediguard_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
    chart_path = pdf_path.replace(".pdf",".png")
    make_chart(report["interactions"], chart_path)

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial","B",16)
    pdf.cell(0,10,"MediGuard AI - Prescription Report",ln=True,align="C")
    pdf.set_font("Arial",size=10)
    pdf.cell(0,8,f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",ln=True)
    pdf.ln(5)

    pdf.set_font("Arial","B",12)
    pdf.cell(0,8,"Patient Details",ln=True)
    pdf.set_font("Arial",size=10)
    pdf.multi_cell(0,5,f"Age: {report['age']}")
    pdf.ln(3)

    pdf.set_font("Arial","B",12)
    pdf.cell(0,8,"Extracted Medications",ln=True)
    pdf.set_font("Arial",size=10)
    for m in report["extracted_medications"]:
        pdf.multi_cell(0,5,f"- {m['name']} | {m['dosage']} | {m['frequency']}")
    pdf.ln(3)

    pdf.set_font("Arial","B",12)
    pdf.cell(0,8,"Interactions",ln=True)
    pdf.set_font("Arial",size=10)
    for i in report["interactions"]:
        pdf.multi_cell(0,5,f"{i['pair']} — {i['severity'].capitalize()}\n{i['description']}\nSafer: {i['safer_alternative']}\n")

    pdf.add_page()
    pdf.set_font("Arial","B",12)
    pdf.cell(0,8,"Clinical Summary",ln=True)
    pdf.set_font("Arial",size=10)
    pdf.multi_cell(0,5,report["summary_text"])
    pdf.ln(3)
    pdf.image(chart_path, x=30, w=150)
    pdf.output(pdf_path)
    return pdf_path


# ------------------------------------------------------
# Core pipeline + UI
# ------------------------------------------------------

def analyze_prescription(age, text):
    meds = extract_medications(text)
    interactions = check_interactions(meds)
    summary = analyze_with_granite(age, meds, interactions)
    confidence = compute_confidence(interactions)
    table = [[i["pair"], i["severity"], i["description"], i["safer_alternative"]] for i in interactions]
    report = {
        "project_name":"MediGuard AI",
        "age":age,
        "extracted_medications":meds,
        "interactions":interactions,
        "confidence_score":confidence,
        "summary_text":summary
    }
    return summary, table, f"{confidence*100:.1f}%", json.dumps(report, indent=2)


def app_interface(age, text):
    summary, table, confidence, report = analyze_prescription(age, text)
    disclaimer = "⚠️ For educational use only. Not a substitute for medical advice."
    return summary, table, confidence, report, disclaimer


custom_css = """
.gradio-container {font-family: 'Inter', sans-serif;}
h1 {color:#1E88E5;text-align:center;}
th {background:#1976D2;color:white;}
"""

with gr.Blocks(css=custom_css, theme="soft") as demo:
    gr.Markdown("# 🩺 MediGuard AI — Prescription Verification System")
    gr.Markdown("Granite-powered NLP + Rule-based Engine for Safe Drug Analysis")

    with gr.Row():
        age = gr.Number(label="Patient Age", value=45)
        text = gr.Textbox(label="Prescription Text", lines=4,
                          placeholder="Enter prescription or doctor's note...")

    analyze_btn = gr.Button("🔍 Analyze Prescription", variant="primary")

    with gr.Tab("Summary & Recommendations"):
        summary_box = gr.Textbox(label="AI Summary", lines=8)
        conf_box = gr.Textbox(label="Confidence", lines=1)
        disclaimer_box = gr.Markdown()

    with gr.Tab("Detailed Interactions"):
        table = gr.Dataframe(headers=["Drug Pair","Severity","Description","Safer Alternative"])

    with gr.Tab("Downloadable Report"):
        json_box = gr.Textbox(label="Structured JSON", lines=15)
        download_btn = gr.DownloadButton(label="⬇️ Download PDF Report")

    analyze_btn.click(fn=app_interface, inputs=[age,text],
                      outputs=[summary_box, table, conf_box, json_box, disclaimer_box])



    def _pdf(json_text):
        """Generate PDF and return it as downloadable bytes for Gradio v4+."""
        pdf_path = generate_pdf(json_text)
        # Read file as bytes and send directly to Gradio
        with open(pdf_path, "rb") as f:
            pdf_bytes = f.read()
        return (pdf_bytes, "mediguard_report.pdf")  # (file data, filename)

    download_btn.click(fn=_pdf, inputs=[json_box], outputs=[download_btn])


demo.launch(share=True, debug=False)


✅ Granite pipeline detected and ready.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://928b451723a0ae54e3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
# Install OCR dependencies
!apt-get install -y tesseract-ocr
!pip install pytesseract Pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [18]:
# ==========================================================
# 🩺 MediGuard AI 2.2 — Stable OCR + Granite + Dashboard + PDF
# ==========================================================

import os, re, json
from io import BytesIO
from datetime import datetime

# Core imports
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
from PIL import Image
from fpdf import FPDF
import pytesseract

DEBUG = False  # change to True for debugging

# -----------------------------
# Check prerequisites
# -----------------------------
if "gen" not in globals():
    raise SystemExit("❌ Granite model pipeline `gen` not found. Load it first.")

DATA_PATH = "data/drug_interactions.csv"
if not os.path.exists(DATA_PATH):
    raise SystemExit("❌ data/drug_interactions.csv not found. Create it before running this.")

df = pd.read_csv(DATA_PATH)
df['drug_a'] = df['drug_a'].str.lower()
df['drug_b'] = df['drug_b'].str.lower()

# -----------------------------
# Utility functions
# -----------------------------
def safe_json_extract(text):
    """Extract last valid JSON object from model output."""
    try:
        matches = list(re.finditer(r"\{(?:[^{}]|\{[^{}]*\})*\}", str(text), re.DOTALL))
        if not matches:
            return None
        block = matches[-1].group(0)
        return json.loads(block)
    except Exception as e:
        if DEBUG: print("safe_json_extract failed:", e)
        return None

def fallback_regex_extract(text):
    """Regex fallback if LLM JSON fails."""
    meds = []
    pattern = r"([A-Za-z][A-Za-z\-]{1,40})\s+(\d+(?:\.\d+)?\s*(?:mg|mcg|g|ml|IU))\s*(\b(OD|BD|TID|QID|daily)\b)?"
    for match in re.finditer(pattern, text, re.IGNORECASE):
        name, dose, freq = match.group(1), match.group(2), match.group(3) or ""
        meds.append({"name": name, "dosage": dose, "frequency": freq})
    return meds

def extract_medications(text):
    """LLM-based JSON extraction with graceful fallback."""
    if not text.strip():
        return []
    prompt = f"""
    Extract structured medication information from this text.
    Return ONLY JSON in this format:
    {{"medications":[{{"name":"DrugName","dosage":"DosageAmount","frequency":"Frequency"}}]}}
    Text: {text}
    """
    try:
        out = gen(prompt, max_new_tokens=200, do_sample=False)[0]["generated_text"]
        js = safe_json_extract(out)
        if js and "medications" in js:
            return js["medications"]
        else:
            return fallback_regex_extract(text)
    except Exception as e:
        if DEBUG: print("extract_medications failed:", e)
        return fallback_regex_extract(text)

def check_interactions(meds):
    """Match drug pairs in dataset."""
    interactions = []
    for i in range(len(meds)):
        for j in range(i+1, len(meds)):
            a, b = meds[i]["name"].lower(), meds[j]["name"].lower()
            match = df[((df["drug_a"] == a) & (df["drug_b"] == b)) |
                       ((df["drug_a"] == b) & (df["drug_b"] == a))]
            if not match.empty:
                row = match.iloc[0]
                interactions.append({
                    "pair": f"{a} + {b}",
                    "severity": row["severity"],
                    "description": row["description"],
                    "safer_alternative": row["safer_alternative"]
                })
            else:
                interactions.append({
                    "pair": f"{a} + {b}",
                    "severity": "unknown",
                    "description": "No documented interaction found.",
                    "safer_alternative": "-"
                })
    return interactions

def analyze_with_granite(age, meds, interactions):
    """Generate clinical summary from Granite."""
    try:
        meds_str = ", ".join([m["name"] for m in meds])
        prompt = f"""
        You are an expert clinical pharmacist.
        A {age}-year-old patient is prescribed: {meds_str}.
        Interactions: {json.dumps(interactions)}.
        Provide a concise summary:
        - Risk level
        - Key interactions and reasons
        - Safer alternatives
        - Final advice (2 lines)
        """
        result = gen(prompt, max_new_tokens=200, do_sample=False)[0]["generated_text"]
        return result.strip()
    except Exception as e:
        if DEBUG: print("analyze_with_granite failed:", e)
        return "Error generating summary."

def make_chart(interactions):
    """Generate pie chart and return PIL image."""
    counts = {"major":0,"moderate":0,"safe":0,"unknown":0}
    for i in interactions:
        counts[i["severity"]] = counts.get(i["severity"], 0) + 1
    labels = [f"{k} ({v})" for k,v in counts.items() if v>0]
    sizes = [v for v in counts.values() if v>0]
    colors = ["#e53935","#fb8c00","#43a047","#bdbdbd"]
    fig, ax = plt.subplots(figsize=(4,3))
    if sizes:
        ax.pie(sizes, labels=labels, autopct="%1.0f%%", colors=colors[:len(sizes)])
    else:
        ax.text(0, 0, "No interactions", ha="center", va="center")
    buf = BytesIO()
    plt.savefig(buf, format="png", bbox_inches="tight")
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)

def compute_confidence(interactions):
    base = 0.95
    penalty = 0.05 * len([i for i in interactions if i["severity"] in ["major","moderate"]])
    return max(0.75, round(base - penalty, 2))

def generate_pdf(report):
    """Generate simple PDF report."""
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", "B", 16)
    pdf.cell(0, 10, "MediGuard AI - Report", ln=True, align="C")
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 8, f"Age: {report['age']}")
    pdf.multi_cell(0, 8, f"Confidence: {report['confidence_score']*100:.1f}%")
    pdf.ln(5)
    pdf.multi_cell(0, 8, "Medications:")
    for m in report["extracted_medications"]:
        pdf.multi_cell(0, 6, f"  - {m['name']} ({m['dosage']}, {m['frequency']})")
    pdf.ln(4)
    pdf.multi_cell(0, 8, "Interactions:")
    for i in report["interactions"]:
        pdf.multi_cell(0, 6, f"{i['pair']} ({i['severity']}): {i['description']}")
    pdf.ln(4)
    pdf.multi_cell(0, 8, "Summary:")
    pdf.multi_cell(0, 6, report["summary_text"])
    path = "/tmp/prescription_report.pdf"
    pdf.output(path)
    return path

def ocr_extract(image):
    if image is None: return ""
    try:
        img = Image.open(image)
        text = pytesseract.image_to_string(img)
        return text.strip()
    except Exception as e:
        return f"OCR error: {e}"

def analyze_prescription(age, text):
    meds = extract_medications(text)
    if not meds:
        return "No valid drugs detected.", [], "0%", None, json.dumps({})
    interactions = check_interactions(meds)
    summary = analyze_with_granite(age, meds, interactions)
    confidence = compute_confidence(interactions)
    chart = make_chart(interactions)
    table = [
        [i["pair"], i["severity"], i["description"], i["safer_alternative"]]
        for i in interactions
    ]
    report = {
        "age": age,
        "extracted_medications": meds,
        "interactions": interactions,
        "confidence_score": confidence,
        "summary_text": summary
    }
    return summary, table, f"{confidence*100:.1f}%", chart, json.dumps(report, indent=2)

def download_pdf(report_json):
    try:
        report = json.loads(report_json)
        path = generate_pdf(report)
        return path
    except Exception as e:
        return None

# -----------------------------
# Gradio Interface
# -----------------------------
with gr.Blocks(theme="soft") as demo:
    gr.Markdown("# 🩺 MediGuard AI 2.2 — Visual Prescription Safety Assistant")

    with gr.Row():
        img_in = gr.Image(type="filepath", label="📷 Upload Prescription Image")
        text_in = gr.Textbox(label="Extracted / Manual Prescription Text", lines=4)
        ocr_btn = gr.Button("🪄 Extract Text via OCR")

    ocr_btn.click(fn=ocr_extract, inputs=[img_in], outputs=[text_in])

    age = gr.Number(label="Patient Age", value=45)
    analyze_btn = gr.Button("🔍 Analyze Prescription", variant="primary")

    with gr.Tab("Summary & Recommendations"):
        summary_out = gr.Textbox(label="AI Clinical Summary", lines=8)
        confidence_out = gr.Textbox(label="Confidence Score", lines=1)

    with gr.Tab("Detailed Interactions"):
        table_out = gr.Dataframe(headers=["Drug Pair", "Severity", "Description", "Safer Alternative"])

    with gr.Tab("Visualization & Report"):
        chart_out = gr.Image(label="Interaction Severity Distribution")
        json_out = gr.Textbox(label="JSON Report", lines=10)
        pdf_btn = gr.DownloadButton(label="⬇️ Download PDF Report")

    analyze_btn.click(fn=analyze_prescription,
                      inputs=[age, text_in],
                      outputs=[summary_out, table_out, confidence_out, chart_out, json_out])

    pdf_btn.click(fn=download_pdf, inputs=[json_out], outputs=[pdf_btn])

demo.launch(share=True, debug=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d835aa496b8a34e3f7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
